In [ ]:
!pip install keras

In [ ]:
!pip install tensorflow


  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, GRU, LSTM, Bidirectional, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
file_path = 'urdu-sentiment-corpus-v1.tsv'
df = pd.read_csv(file_path, delimiter='\t')
print(df.head())

                                               Tweet Class
0  میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...     P
1  چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...     N
2                           ٹویٹر کا خیال کیسے آیا ؟     O
3  سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...     P
4    ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار :أْ     P


In [ ]:
df['Tweet'] = df['Tweet'].str.lower()
print(df.head())

                                               Tweet Class
0  میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...     P
1  چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...     N
2                           ٹویٹر کا خیال کیسے آیا ؟     O
3  سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...     P
4    ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار :أْ     P


In [ ]:
X = df['Tweet']
y = df['Class']

In [ ]:
y_binary = np.where(y == 'P', 1, 0)

In [ ]:
if np.any((y_binary != 0) & (y_binary != 1)):
    raise ValueError("Labels contain mixed or unknown targets.")

In [ ]:
missing_labels = np.isnan(y_binary)
if np.any(missing_labels):
    raise ValueError("Missing values found in labels.")

In [ ]:
y_binary = np.expand_dims(y_binary, axis=-1)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
max_sequence_length = 100
X = pad_sequences(X, maxlen=max_sequence_length)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.25, random_state=42)

In [ ]:
def create_model(model_type, num_layers, dropout_rate):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_sequence_length))
    if model_type == 'RNN':
        for _ in range(num_layers):
            model.add(SimpleRNN(64, return_sequences=True))
    elif model_type == 'GRU':
        for _ in range(num_layers):
            model.add(GRU(64, return_sequences=True))
    elif model_type == 'LSTM':
        for _ in range(num_layers):
            model.add(LSTM(64, return_sequences=True))
    elif model_type == 'BiLSTM':
        for _ in range(num_layers):
            model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [ ]:
results = []
num_layers_options = [2, 3]
dropout_rate_options = [0.3, 0.7]

In [ ]:
for model_type in ['RNN', 'GRU', 'LSTM', 'BiLSTM']:
    for num_layers in num_layers_options:
        for dropout_rate in dropout_rate_options:
            model = create_model(model_type, num_layers, dropout_rate)
            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
            model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
            y_pred_prob = model.predict(X_test)
            y_pred_mean = np.mean(y_pred_prob, axis=1)
            y_pred = (y_pred_mean > 0.5).astype(int)
            if y_test.shape != y_pred.shape:
                raise ValueError("Shapes of y_test and y_pred do not match.")
            if np.isnan(y_test).any():
                raise ValueError("Missing values found in y_test.")
            if y_test.shape != y_pred.shape:
                raise ValueError("Shapes of y_test and y_pred do not match.")
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            results.append({
                'Model': model_type,
                'Num Layers': num_layers,
                'Dropout Rate': dropout_rate,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1-score': f1
            })

8/8 [==============================] - 1s 21ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


8/8 [==============================] - 4s 155ms/step


In [ ]:
results_df = pd.DataFrame(results)
print(results_df)

     Model  Num Layers  Dropout Rate  Accuracy  Precision    Recall  F1-score
0      RNN           2           0.3     0.512   1.000000  0.039370  0.075758
1      RNN           2           0.7     0.552   0.532751  0.960630  0.685393
2      RNN           3           0.3     0.492   0.000000  0.000000  0.000000
3      RNN           3           0.7     0.500   1.000000  0.015748  0.031008
4      GRU           2           0.3     0.512   0.510288  0.976378  0.670270
5      GRU           2           0.7     0.616   0.618321  0.637795  0.627907
6      GRU           3           0.3     0.596   0.569149  0.842520  0.679365
7      GRU           3           0.7     0.580   0.677419  0.330709  0.444444
8     LSTM           2           0.3     0.572   0.621951  0.401575  0.488038
9     LSTM           2           0.7     0.564   0.673077  0.275591  0.391061
10    LSTM           3           0.3     0.512   0.608696  0.110236  0.186667
11    LSTM           3           0.7     0.556   0.766667  0.181

**QUESTION-2**

In [ ]:
!pip install --upgrade keras

In [ ]:
!pip install tensorflow


  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1


In [ ]:
from gensim.models import KeyedVectors
from keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim.models import KeyedVectors
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [ ]:
word2vec_model_path = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin'
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True, limit=20000)

In [ ]:
file_path = 'urdu-sentiment-corpus-v1.tsv'
df = pd.read_csv(file_path, delimiter='\t')

In [ ]:
X = df['Tweet']
y = df['Class']
y_binary = np.where(y == 'P', 1, 0)

In [ ]:
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

In [ ]:
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_binary, test_size=0.25, random_state=42)

In [ ]:
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=300, weights=[embedding_matrix],
                    input_length=max_sequence_length, trainable=False))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.7))
model.add(LSTM(64))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)
y_pred_prob = model.predict(X_test)

Epoch 1/10
12/12 [==============================] - 12s 400ms/step - loss: 0.6928 - accuracy: 0.5253
Epoch 2/10
12/12 [==============================] - 5s 437ms/step - loss: 0.6919 - accuracy: 0.5293
Epoch 3/10
12/12 [==============================] - 3s 200ms/step - loss: 0.6914 - accuracy: 0.5280
Epoch 4/10
12/12 [==============================] - 2s 196ms/step - loss: 0.6930 - accuracy: 0.5307
Epoch 5/10
12/12 [==============================] - 2s 199ms/step - loss: 0.6922 - accuracy: 0.5240
Epoch 6/10
12/12 [==============================] - 2s 196ms/step - loss: 0.6922 - accuracy: 0.5293
Epoch 7/10
12/12 [==============================] - 4s 309ms/step - loss: 0.6904 - accuracy: 0.5267
Epoch 8/10
12/12 [==============================] - 3s 237ms/step - loss: 0.6919 - accuracy: 0.5253
Epoch 9/10
12/12 [==============================] - 2s 196ms/step - loss: 0.6933 - accuracy: 0.5293
Epoch 10/10
8/8 [==============================] - 1s 52ms/step


In [ ]:
threshold = 0.5
y_pred = (y_pred_prob > threshold).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
results = []
for threshold in np.arange(0.1, 0.5, 0.1):
    y_pred = (y_pred_prob > threshold).astype(int)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    results.append({'Threshold': threshold, 'Precision': precision, 'Recall': recall, 'F1-score': f1})

In [ ]:
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
results_df = pd.DataFrame(results)
print(results_df)

Accuracy: 0.492
Precision: 0.508
Recall: 1.0
F1-score: 0.6737400530503979
   Threshold  Precision  Recall  F1-score
0        0.1      0.508     1.0   0.67374
1        0.2      0.508     1.0   0.67374
2        0.3      0.508     1.0   0.67374
3        0.4      0.508     1.0   0.67374


In [ ]:
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index


In [ ]:
glove_embeddings_path = '/content/drive/MyDrive/glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_embeddings_path)

In [ ]:
file_path = 'urdu-sentiment-corpus-v1.tsv'
df = pd.read_csv(file_path, delimiter='\t')

In [ ]:
X = df['Tweet']
y = df['Class']

In [ ]:
y_binary = np.where(y == 'P', 1, 0)

In [ ]:
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

In [ ]:
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_binary, test_size=0.25, random_state=42)

In [ ]:
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
for word, i in tokenizer.word_index.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=300, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(LSTM(64, return_sequences=True))  # First LSTM layer
model.add(Dropout(0.7))
model.add(LSTM(64))  # Second LSTM layer
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

Epoch 1/10
12/12 [==============================] - 8s 271ms/step - loss: 0.6936 - accuracy: 0.5200
Epoch 2/10
12/12 [==============================] - 3s 284ms/step - loss: 0.6904 - accuracy: 0.5480
Epoch 3/10
12/12 [==============================] - 2s 198ms/step - loss: 0.6915 - accuracy: 0.5307
Epoch 4/10
12/12 [==============================] - 2s 195ms/step - loss: 0.6889 - accuracy: 0.5387
Epoch 5/10
12/12 [==============================] - 2s 198ms/step - loss: 0.6898 - accuracy: 0.5320
Epoch 6/10
12/12 [==============================] - 3s 238ms/step - loss: 0.6848 - accuracy: 0.5493
Epoch 7/10
12/12 [==============================] - 4s 344ms/step - loss: 0.6814 - accuracy: 0.5600
Epoch 8/10
12/12 [==============================] - 2s 194ms/step - loss: 0.6831 - accuracy: 0.5640
Epoch 9/10
12/12 [==============================] - 2s 197ms/step - loss: 0.6743 - accuracy: 0.5560
Epoch 10/10
12/12 [==============================] - 2s 198ms/step - loss: 0.6770 - accuracy: 0.5693

In [ ]:
y_pred_prob = model.predict(X_test)

8/8 [==============================] - 3s 97ms/step


In [ ]:
threshold = 0.5
y_pred = (y_pred_prob > threshold).astype(int)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [ ]:
results = []
for threshold in np.arange(0.1, 0.6, 0.1):
    y_pred = (y_pred_prob > threshold).astype(int)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    results.append({'Threshold': threshold, 'Precision': precision, 'Recall': recall, 'F1-score': f1})

In [ ]:
print("Evaluation metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
results_df = pd.DataFrame(results)
print(results_df)


Evaluation metrics:
Accuracy: 0.496
Precision: 0.5454545454545454
Recall: 0.047244094488188976
F1-score: 0.08695652173913043
   Threshold  Precision    Recall  F1-score
0        0.1   0.508000  1.000000  0.673740
1        0.2   0.508000  1.000000  0.673740
2        0.3   0.508000  1.000000  0.673740
3        0.4   0.508696  0.921260  0.655462
4        0.5   0.545455  0.047244  0.086957


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim.models import KeyedVectors
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
fasttext_model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/wiki-news-300d-1M.vec', binary=False, limit=20000)

In [ ]:
file_path = 'urdu-sentiment-corpus-v1.tsv'
df = pd.read_csv(file_path, delimiter='\t')

In [ ]:
X = df['Tweet']
y = df['Class']
y_binary = np.where(y == 'P', 1, 0)

In [ ]:
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_binary, test_size=0.25, random_state=42)

In [ ]:
embedding_dim = 300
num_words = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in fasttext_model:
        embedding_matrix[i] = fasttext_model[word]

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=300, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.7))
model.add(LSTM(64))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

Epoch 1/10
12/12 [==============================] - 8s 195ms/step - loss: 0.6925 - accuracy: 0.5293
Epoch 2/10
12/12 [==============================] - 4s 314ms/step - loss: 0.6916 - accuracy: 0.5307
Epoch 3/10
12/12 [==============================] - 3s 227ms/step - loss: 0.6917 - accuracy: 0.5293
Epoch 4/10
12/12 [==============================] - 2s 193ms/step - loss: 0.6931 - accuracy: 0.5293
Epoch 5/10
12/12 [==============================] - 2s 193ms/step - loss: 0.6912 - accuracy: 0.5187
Epoch 6/10
12/12 [==============================] - 2s 193ms/step - loss: 0.6916 - accuracy: 0.5133
Epoch 7/10
12/12 [==============================] - 3s 263ms/step - loss: 0.6927 - accuracy: 0.5160
Epoch 8/10
12/12 [==============================] - 3s 274ms/step - loss: 0.6931 - accuracy: 0.5280
Epoch 9/10
12/12 [==============================] - 2s 191ms/step - loss: 0.6926 - accuracy: 0.5187
Epoch 10/10
12/12 [==============================] - 2s 194ms/step - loss: 0.6911 - accuracy: 0.5333

In [ ]:
y_pred_prob = model.predict(X_test)

8/8 [==============================] - 1s 53ms/step


In [ ]:
threshold = 0.5
y_pred = (y_pred_prob > threshold).astype(int)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [ ]:
print("Evaluation metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Evaluation metrics:
Accuracy: 0.496
Precision: 1.0
Recall: 0.007874015748031496
F1-score: 0.015625


In [ ]:
results = []
for threshold in np.arange(0.1, 0.6, 0.1):
    y_pred = (y_pred_prob > threshold).astype(int)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    results.append({'Threshold': threshold, 'Precision': precision, 'Recall': recall, 'F1-score': f1})

In [ ]:
results_df = pd.DataFrame(results)
print(results_df)


   Threshold  Precision    Recall  F1-score
0        0.1      0.508  1.000000  0.673740
1        0.2      0.508  1.000000  0.673740
2        0.3      0.508  1.000000  0.673740
3        0.4      0.508  1.000000  0.673740
4        0.5      1.000  0.007874  0.015625


In [ ]:
#ELMO
!pip install allennlp


In [ ]:
!pip show allennlp


Name: allennlp
Version: 2.10.1
Summary: An open-source NLP research library, built on PyTorch.
Home-page: https://github.com/allenai/allennlp
Author: Allen Institute for Artificial Intelligence
Author-email: allennlp@allenai.org
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: base58, cached-path, dill, fairscale, filelock, h5py, huggingface-hub, jsonnet, lmdb, more-itertools, nltk, numpy, protobuf, pytest, requests, sacremoses, scikit-learn, scipy, sentencepiece, spacy, tensorboardX, termcolor, torch, torchvision, tqdm, traitlets, transformers, typer, wandb
Required-by: 


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from allennlp.modules.elmo import Elmo, batch_to_ids
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [ ]:
file_path = 'urdu-sentiment-corpus-v1.tsv'
df = pd.read_csv(file_path, delimiter='\t')

In [ ]:
X = df['Tweet']
y = df['Class']
y_binary = np.where(y == 'P', 1, 0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.25, random_state=42)

In [ ]:
options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo = Elmo(options_file, weight_file, num_output_representations=1, dropout=0)

In [ ]:
def get_elmo_embeddings(sentences, max_length=50):
    character_ids = batch_to_ids([sentence.split() for sentence in sentences])
    embeddings = elmo(character_ids)['elmo_representations'][0].detach().numpy()
    padded_embeddings = np.zeros((len(sentences), max_length, embeddings.shape[2]))
    for i, embedding in enumerate(embeddings):
        if embedding.shape[0] < max_length:
            padded_embeddings[i, :embedding.shape[0], :] = embedding
        else:
            padded_embeddings[i, :, :] = embedding[:max_length, :]
    return padded_embeddings

In [ ]:
X_train_elmo = get_elmo_embeddings(X_train)
X_test_elmo = get_elmo_embeddings(X_test)

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_elmo.shape[1], X_train_elmo.shape[2])))
model.add(Dropout(0.7))
model.add(Flatten())  # Add Flatten layer to convert 3D output to 2D
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_elmo, y_train, epochs=10, batch_size=64, verbose=1)

Epoch 1/10
12/12 [==============================] - 2s 39ms/step - loss: 0.7460 - accuracy: 0.4907
Epoch 2/10
12/12 [==============================] - 0s 39ms/step - loss: 0.6931 - accuracy: 0.5547
Epoch 3/10
12/12 [==============================] - 0s 36ms/step - loss: 0.6774 - accuracy: 0.5787
Epoch 4/10
12/12 [==============================] - 0s 37ms/step - loss: 0.6658 - accuracy: 0.6080
Epoch 5/10
12/12 [==============================] - 0s 38ms/step - loss: 0.6507 - accuracy: 0.6333
Epoch 6/10
12/12 [==============================] - 0s 36ms/step - loss: 0.6323 - accuracy: 0.6627
Epoch 7/10
12/12 [==============================] - 0s 37ms/step - loss: 0.6282 - accuracy: 0.6640
Epoch 8/10
12/12 [==============================] - 0s 35ms/step - loss: 0.6060 - accuracy: 0.6680
Epoch 9/10
12/12 [==============================] - 0s 37ms/step - loss: 0.5921 - accuracy: 0.7053
Epoch 10/10
12/12 [==============================] - 0s 35ms/step - loss: 0.5808 - accuracy: 0.6893


In [ ]:
loss, accuracy = model.evaluate(X_test_elmo, y_test, verbose=0)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.5239999890327454


In [ ]:
y_pred_prob = model.predict(X_test_elmo)
y_pred = (y_pred_prob > 0.5).astype(int)

8/8 [==============================] - 0s 9ms/step


In [ ]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [ ]:
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.532258064516129
Recall: 0.5196850393700787
F1-score: 0.5258964143426295
